# Creating charts and visualisations in Python

1. Addressing our new and old data sets
    1. Wrangling, 
    2. Cleaning, and 
    3. Merging 
2. Creating visualisations (included in separate notebook)

In [1]:
# Importing libraries
from downcast import reduce
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Path
path = r"C:\Users\Natalia\Documents\Instacart Basket Analysis"

In [3]:
#import customers data set
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col=False)

In [4]:
#checking output
print('Sample of new customers data set')
df.head(10)

Sample of new customers data set


,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


In [5]:
print('Number of rows and columns in customers data set')
df.shape

Number of rows and columns in customers data set


(206209, 10)

##### Initial observations:

Our new customers data set provides personal information about Instacart's customer base including first name, surname, gender, state location, age, when they joined Instacart, the number of dependents they have, their family status, and finally their income. Already there are some spelling and syntactical errors in the column names which we will be able to fix to make the data set easier to understand. 

There are 206,209 rows, so lot's of customers! 

### A. Data Wrangling

#### Approach

1. Some simple exploratory analysis to understand a couple of the columns.
2. Removal of the first_name, and surname columns. User_id is a more useful customer identifier than first or surname.  
3. Amend any spelling and syntactical errors in the column titles. This will include use of capital and lowercase letters, and use of underscores and spaces. 
4. Check data types of columns to ensure they are correct and appropriate for the type of data collected
5. Use a data dictionary to change the data within the STATE column to two-letter codes. Although not specifically needed, reducing the data to two-letters is cleaner, and would make it easier to, for example, search for specific states using the isin fucntion. 
6. Create a new column with flag related to whether or not the customer has dependents. Instacart could provide targeted advertising to individuals based on whether they have dependents on not.


##### 1. Initial exploratory analysis to understand some of the columns

In [6]:
#checking gender column
df['Gender'].value_counts()

Male      104067
Female    102142
Name: Gender, dtype: int64

In [7]:
#checking age column
df['Age'].value_counts()

19    3329
55    3317
51    3317
56    3306
32    3305
      ... 
65    3145
25    3127
66    3114
50    3102
36    3101
Name: Age, Length: 64, dtype: int64

In [8]:
df['Age'].describe()

count    206209.000000
mean         49.501646
std          18.480962
min          18.000000
25%          33.000000
50%          49.000000
75%          66.000000
max          81.000000
Name: Age, dtype: float64

In [9]:
#checking fam_status column
df['fam_status'].value_counts()

married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: fam_status, dtype: int64

In [10]:
#checking income column
df['income'].describe()

count    206209.000000
mean      94632.852548
std       42473.786988
min       25903.000000
25%       59874.000000
50%       93547.000000
75%      124244.000000
max      593901.000000
Name: income, dtype: float64

In [11]:
#checking n_dependants
df['n_dependants'].describe()

count    206209.000000
mean          1.499823
std           1.118433
min           0.000000
25%           0.000000
50%           1.000000
75%           3.000000
max           3.000000
Name: n_dependants, dtype: float64

##### Observations:

I wanted to understand more about the types of values contained within these columns before I started amending anything. Everything so far seems reasonable. 

##### 2. Removal of first name and surname columns

In [12]:
#remove date_joined column using drop function
df.drop(columns = ['First Name', 'Surnam'])

,user_id,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374
...,...,...,...,...,...,...,...,...
206204,168073,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Female,California,27,4/1/2020,1,married,99799


In [13]:
#define dateframe minus date_joined column
df = df.drop(columns = ['First Name', 'Surnam'])

In [14]:
#check output
df.head()

,user_id,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [15]:
#checking shape
df.shape

(206209, 8)

##### 3. Amending spelling and syntax errors in columns names

In [16]:
#amending First Name, Surnam, Gender, STATE, and Age columns
df.rename(columns = {'Gender':'gender', 'STATE':'state', 'Age':'age'}, inplace = True)

In [17]:
df.head()

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


##### 4. Checking datatypes

In [18]:
#returning the datatypes for all columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   gender        206209 non-null  object
 2   state         206209 non-null  object
 3   age           206209 non-null  int64 
 4   date_joined   206209 non-null  object
 5   n_dependants  206209 non-null  int64 
 6   fam_status    206209 non-null  object
 7   income        206209 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 12.6+ MB


###### Observations:

Whilst the majority of the datatypes themselves are all correct, we should reduce the size of some of them to save on space. The date_joined column should be changed to datetime. 

In [19]:
#amending datatype for user_id, age, n_dependents, and income
df['user_id']=df['user_id'].astype('int32')
df['age'] =df['age'].astype('int8')
df['date_joined'] = df['date_joined'].apply(pd.to_datetime)
df['n_dependants']=df['n_dependants'].astype('int16')
df['income']=df['income'].astype('int32')

In [20]:
#checking output
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   user_id       206209 non-null  int32         
 1   gender        206209 non-null  object        
 2   state         206209 non-null  object        
 3   age           206209 non-null  int8          
 4   date_joined   206209 non-null  datetime64[ns]
 5   n_dependants  206209 non-null  int16         
 6   fam_status    206209 non-null  object        
 7   income        206209 non-null  int32         
dtypes: datetime64[ns](1), int16(1), int32(2), int8(1), object(3)
memory usage: 8.5+ MB


In [21]:
df.head()

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income
0,26711,Female,Missouri,48,2017-01-01,3,married,165665
1,33890,Female,New Mexico,36,2017-01-01,0,single,59285
2,65803,Male,Idaho,35,2017-01-01,2,married,99568
3,125935,Female,Iowa,40,2017-01-01,0,single,42049
4,130797,Female,Maryland,26,2017-01-01,1,married,40374


##### 5. Using a data dictionary to change state names to two-letter abbreviations

In [22]:
#creating list of all state names
states_listed = df['state'].unique()

In [23]:
#checking output
print(sorted(states_listed))

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


In [24]:
#number of each state
df['state'].value_counts().sort_index()

Alabama                 4044
Alaska                  4044
Arizona                 4044
Arkansas                4044
California              4044
Colorado                4044
Connecticut             4044
Delaware                4044
District of Columbia    4044
Florida                 4044
Georgia                 4044
Hawaii                  4044
Idaho                   4044
Illinois                4044
Indiana                 4044
Iowa                    4044
Kansas                  4043
Kentucky                4043
Louisiana               4043
Maine                   4043
Maryland                4043
Massachusetts           4043
Michigan                4043
Minnesota               4043
Mississippi             4043
Missouri                4043
Montana                 4043
Nebraska                4043
Nevada                  4043
New Hampshire           4043
New Jersey              4043
New Mexico              4043
New York                4043
North Carolina          4043
North Dakota  

In [25]:
#counting number of states
state_count = df['state'].nunique()
print(state_count)

51


In [26]:
#creating data dictionary, abbreviations were retrieved from US government website
state_dict = {'Alabama': 'AL',
              'Alaska': 'AK', 
              'Arizona': 'AZ',
              'Arkansas': 'AR',
              'California': 'CA',
              'Colorado': 'CO',
              'Connecticut': 'CT',
              'Delaware': 'DE',
              'District of Columbia': 'DC',
              'Florida': 'FL',
              'Georgia': 'GA',
              'Hawaii': 'HI',
              'Idaho': 'ID',
              'Illinois': 'IL',
              'Indiana': 'IN',
              'Iowa': 'IA',
              'Kansas': 'KS',
              'Kentucky': 'KY',
              'Louisiana': 'LA',
              'Maine': 'ME',
              'Maryland': 'MD',
              'Massachusetts': 'MA',
              'Michigan': 'MI',
              'Minnesota': 'MN',
              'Mississippi': 'MS',
              'Missouri': 'MO',
              'Montana': 'MT',
              'Nebraska': 'NE',
              'Nevada': 'NV',
              'New Hampshire': 'NH',
              'New Jersey': 'NJ',
              'New Mexico': 'NM',
              'New York': 'NY',
              'North Carolina': 'NC',
              'North Dakota': 'ND',
              'Ohio': 'OH',
              'Oklahoma': 'OK',
              'Oregon': 'OR',
              'Pennsylvania': 'PA',
              'Rhode Island': 'RI',
              'South Carolina': 'SC',
              'South Dakota': 'SD',
              'Tennessee': 'TN',
              'Texas': 'TX',
              'Utah': 'UT',
              'Vermont': 'VT',
              'Virginia': 'VA',
              'Washington': 'WA',
              'West Virginia': 'WV',
              'Wisconsin': 'WI',
              'Wyoming': 'WY'}

In [27]:
#replace values in state column with state dictionary 
df2 = df.replace({'state': state_dict})

In [28]:
df2.sample(10)

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income
163949,135606,Male,WA,65,2019-08-04,2,married,33466
95666,3812,Male,MO,51,2018-07-04,1,married,30996
82090,79413,Male,CO,64,2018-04-17,0,divorced/widowed,53580
13103,108338,Male,IL,51,2017-03-17,0,single,114148
61406,96319,Female,OK,29,2017-12-20,2,married,98468
30006,97702,Female,MS,26,2017-06-23,1,married,28779
2627,31425,Female,DC,68,2017-01-16,3,married,140316
64246,4640,Male,WI,47,2018-01-05,3,married,45422
55377,118286,Male,KS,42,2017-11-15,2,married,63789
140148,47318,Female,SC,72,2019-03-19,3,married,50618


In [29]:
df2['state'].value_counts().sort_index()

AK    4044
AL    4044
AR    4044
AZ    4044
CA    4044
CO    4044
CT    4044
DC    4044
DE    4044
FL    4044
GA    4044
HI    4044
IA    4044
ID    4044
IL    4044
IN    4044
KS    4043
KY    4043
LA    4043
MA    4043
MD    4043
ME    4043
MI    4043
MN    4043
MO    4043
MS    4043
MT    4043
NC    4043
ND    4043
NE    4043
NH    4043
NJ    4043
NM    4043
NV    4043
NY    4043
OH    4043
OK    4043
OR    4043
PA    4043
RI    4043
SC    4043
SD    4043
TN    4043
TX    4043
UT    4043
VA    4043
VT    4043
WA    4043
WI    4043
WV    4043
WY    4043
Name: state, dtype: int64

The value counts for the new column matches those for the state column in the original dataframe

##### 6. Create a new flag column on whether the customer has dependants 

In [30]:
df2.loc[df2['n_dependants'] >= 1, 'dependants_loc'] = 'Has dependants'

In [31]:
df2.loc[df2['n_dependants'] < 1, 'dependants_loc'] = 'No dependants'

In [32]:
df2['dependants_loc'].value_counts(dropna = False)

Has dependants    154607
No dependants      51602
Name: dependants_loc, dtype: int64

In [33]:
df2.head()

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income,dependants_loc
0,26711,Female,MO,48,2017-01-01,3,married,165665,Has dependants
1,33890,Female,NM,36,2017-01-01,0,single,59285,No dependants
2,65803,Male,ID,35,2017-01-01,2,married,99568,Has dependants
3,125935,Female,IA,40,2017-01-01,0,single,42049,No dependants
4,130797,Female,MD,26,2017-01-01,1,married,40374,Has dependants


Although the code has worked, I would prefer for my two columns related to dependents to be situated next to one another. 

In [34]:
#creating a list of my column titles
titles = list(df2.columns)
titles

['user_id',
 'gender',
 'state',
 'age',
 'date_joined',
 'n_dependants',
 'fam_status',
 'income',
 'dependants_loc']

In [35]:
#creating a new list, with the new changed order of columns
new_titles = ['user_id',
 'gender',
 'state',
 'age',
 'date_joined',
 'n_dependants',
 'dependants_loc',
 'fam_status',
 'income']

In [36]:
#assigning my new column title order to my dataframe
df3 = df2[new_titles]

In [37]:
#checking my output
df3.sample(5)

,user_id,gender,state,age,date_joined,n_dependants,dependants_loc,fam_status,income
40745,44942,Male,GA,50,2017-08-23,0,No dependants,single,83616
58953,96290,Female,AK,19,2017-12-06,2,Has dependants,living with parents and siblings,52665
96021,1110,Male,PA,51,2018-07-06,1,Has dependants,married,40446
24706,39110,Female,TX,78,2017-05-23,0,No dependants,divorced/widowed,125053
46290,117247,Female,WV,24,2017-09-24,2,Has dependants,married,69401


#### Final check of shape and size of dataframe 

In [38]:
df3.sample(10)

,user_id,gender,state,age,date_joined,n_dependants,dependants_loc,fam_status,income
154260,68122,Male,MS,79,2019-06-08,2,Has dependants,married,32018
6610,109466,Male,ME,48,2017-02-07,2,Has dependants,married,150391
130294,113716,Female,MS,66,2019-01-21,0,No dependants,divorced/widowed,115230
128949,152593,Male,AL,18,2019-01-13,1,Has dependants,living with parents and siblings,77162
44114,183484,Female,MS,36,2017-09-12,2,Has dependants,married,67366
102765,78585,Male,UT,31,2018-08-14,2,Has dependants,married,52110
135121,182612,Male,OR,23,2019-02-18,3,Has dependants,married,74288
144406,92688,Female,MT,62,2019-04-12,3,Has dependants,married,32247
134094,97262,Male,CA,31,2019-02-12,0,No dependants,single,50077
171032,25628,Female,NM,61,2019-09-13,2,Has dependants,married,96076


In [39]:
df3.shape

(206209, 9)

##### Final Observations

In terms of the shape of our dataframe, we have the same number of rows as before. We have dropped three columns, first name, surname, and date_joined, and created another, dependants_loc. 

We have also ensured consistency in our column naming conventions and changed the state names to abbreviations. 

### B. Data Quality and Consistency 

#### Approach 

We're going to be looking at a few things in terms of data quality and consistency 

1. Checking for null values
2. Checking for duplicates
3. Checking for mixed-type data 

1. Checking for null values

In [40]:
df3.isnull().sum()

user_id           0
gender            0
state             0
age               0
date_joined       0
n_dependants      0
dependants_loc    0
fam_status        0
income            0
dtype: int64

##### Observations:

No missing data observed

2. Checking for duplicates

In [41]:
df_dups = df3[df3.duplicated()]

In [42]:
df_dups

,user_id,gender,state,age,date_joined,n_dependants,dependants_loc,fam_status,income


##### Observations:

No duplicate rows found

3. Checking for mixed-type data 

In [43]:
for col in df3.columns.tolist():
    weird = (df3[[col]].applymap(type) != df3[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df3[weird]) > 0:
        print (col)

No mixed-type data identified. 

Interestingly, when I ran this code on the original data there were over 11,000 mixed-type data points in the first name field, representing around 5.5.% of the data set, but given that name is not important to our statistical analysis we have removed the column. 

## C. Merging

Merging our new customers dataset with our existing orders and products dataset

In [44]:
#import orders and products dataset 
df_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'full_merged5.pkl'))

In [45]:
#check output
df_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,prices,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,order_freq,order_freq_flag
0,2539329,1,1,2,8,NaN,196,1,0,both,...,9.0,Mid-range product,Regularly busy,Average orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,both,...,9.0,Mid-range product,Slowest days,Average orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,both,...,9.0,Mid-range product,Slowest days,Most orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,both,...,9.0,Mid-range product,Slowest days,Average orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,both,...,9.0,Mid-range product,Slowest days,Most orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer


In [46]:
df_merge.shape

(32404859, 23)

In [47]:
df_merge.describe()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,prices,max_order,avg_price,order_freq
count,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.032876e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240485e+07
mean,1.710745e+06,1.029372e+05,1.714230e+01,2.738867e+00,1.342515e+01,1.110408e+01,2.559866e+04,8.352547e+00,5.895873e-01,7.119612e+01,9.919792e+00,1.198023e+01,3.305217e+01,1.198023e+01,1.039776e+01
std,9.872988e+05,5.946610e+04,1.753532e+01,2.090077e+00,4.246380e+00,8.779064e+00,1.408400e+04,7.127071e+00,4.919087e-01,3.821139e+01,6.281485e+00,4.956554e+02,2.515525e+01,8.324227e+01,7.131754e+00
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.559470e+05,5.142200e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,1.354400e+04,3.000000e+00,0.000000e+00,3.100000e+01,4.000000e+00,4.200000e+00,1.300000e+01,7.390000e+00,6.000000e+00
50%,1.711049e+06,1.026160e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,2.530200e+04,6.000000e+00,1.000000e+00,8.300000e+01,9.000000e+00,7.400000e+00,2.600000e+01,7.820000e+00,8.000000e+00
75%,2.565499e+06,1.543890e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01,3.794700e+04,1.100000e+01,1.000000e+00,1.070000e+02,1.600000e+01,1.130000e+01,4.700000e+01,8.250000e+00,1.300000e+01
max,3.421083e+06,2.062090e+05,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01,4.968800e+04,1.450000e+02,1.000000e+00,1.340000e+02,2.100000e+01,9.999900e+04,9.900000e+01,2.500542e+04,3.000000e+01


In [48]:
#reminder of our new data set to be merged 
df3.head()

,user_id,gender,state,age,date_joined,n_dependants,dependants_loc,fam_status,income
0,26711,Female,MO,48,2017-01-01,3,Has dependants,married,165665
1,33890,Female,NM,36,2017-01-01,0,No dependants,single,59285
2,65803,Male,ID,35,2017-01-01,2,Has dependants,married,99568
3,125935,Female,IA,40,2017-01-01,0,No dependants,single,42049
4,130797,Female,MD,26,2017-01-01,1,Has dependants,married,40374


In [49]:
df3.shape

(206209, 9)

To merge our two dataframes, we need to have a common variable. In this case, it is user_id.

In [50]:
#merging our datasets
df_combo = df_merge.merge(df3, on = ['user_id'])

In [51]:
# reduce the size of dataframe
df_combo = reduce(df_combo)

In [52]:
df_combo.sample(5)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,order_freq,order_freq_flag,gender,state,age,date_joined,n_dependants,dependants_loc,fam_status,income
21781427,2011418,17798,5,4,13,30.0,25849,21,0,both,...,30.0,Non-frequent customer,Female,WI,24,2020-03-02,2,Has dependants,married,51827
18227762,758299,37846,13,5,23,4.0,47766,3,1,both,...,6.0,Frequent customer,Male,AR,42,2017-07-29,1,Has dependants,married,139096
13214860,1521173,160084,1,1,17,NaN,15290,7,0,both,...,6.0,Frequent customer,Male,VT,37,2018-12-14,3,Has dependants,married,80187
2826198,194753,5955,39,1,16,4.0,47766,2,1,both,...,7.0,Frequent customer,Male,PA,75,2017-12-10,0,No dependants,divorced/widowed,114310
14302488,1993544,176145,4,0,11,2.0,3265,7,1,both,...,2.0,Frequent customer,Female,SD,76,2017-10-09,0,No dependants,divorced/widowed,137989


In [53]:
df_combo.shape

(32404859, 31)

In [54]:
#Checking value counts
df_combo['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [55]:
df_combo.describe()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,prices,max_order,avg_price,order_freq,age,n_dependants,income
count,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,30328763.0,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,32404854.0,3.240486e+07,3.240486e+07,3.240486e+07
mean,1.710745e+06,1.029372e+05,1.714230e+01,2.738867e+00,1.342515e+01,NaN,2.559866e+04,8.352547e+00,5.895873e-01,7.119612e+01,9.919792e+00,1.021125e+01,3.305217e+01,NaN,NaN,4.946527e+01,1.501896e+00,9.943773e+04
std,9.872988e+05,5.946610e+04,1.753532e+01,2.090077e+00,4.246380e+00,0.0,1.408400e+04,7.127071e+00,4.919087e-01,3.821139e+01,6.281485e+00,4.956361e+02,2.515525e+01,NaN,0.0,1.848558e+01,1.118865e+00,4.305727e+04
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.0,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0,1.800000e+01,0.000000e+00,2.590300e+04
25%,8.559470e+05,5.142200e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.0,1.354400e+04,3.000000e+00,0.000000e+00,3.100000e+01,4.000000e+00,4.200000e+00,1.300000e+01,7.390625e+00,6.0,3.300000e+01,1.000000e+00,6.700400e+04
50%,1.711049e+06,1.026160e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.0,2.530200e+04,6.000000e+00,1.000000e+00,8.300000e+01,9.000000e+00,7.400000e+00,2.600000e+01,7.820312e+00,8.0,4.900000e+01,2.000000e+00,9.661800e+04
75%,2.565499e+06,1.543890e+05,2.400000e+01,5.000000e+00,1.600000e+01,15.0,3.794700e+04,1.100000e+01,1.000000e+00,1.070000e+02,1.600000e+01,1.130000e+01,4.700000e+01,8.250000e+00,13.0,6.500000e+01,3.000000e+00,1.279120e+05
max,3.421083e+06,2.062090e+05,9.900000e+01,6.000000e+00,2.300000e+01,30.0,4.968800e+04,1.450000e+02,1.000000e+00,1.340000e+02,2.100000e+01,9.999900e+04,9.900000e+01,2.500800e+04,30.0,8.100000e+01,3.000000e+00,5.939010e+05


In [56]:
#checking on space constraints
df_combo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 31 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   order_id                int32         
 1   user_id                 int32         
 2   order_number            int8          
 3   orders_day_of_week      int8          
 4   order_hour_of_day       int8          
 5   days_since_prior_order  float16       
 6   product_id              int32         
 7   add_to_cart_order       int16         
 8   reordered               int8          
 9   _merge                  category      
 10  product_name            category      
 11  aisle_id                int16         
 12  department_id           int8          
 13  prices                  float32       
 14  price_range_loc         category      
 15  busiest_days            category      
 16  busiest_period_of_day   category      
 17  max_order               int8          
 18  

## Export dataframe

In [57]:
#export to pickle 
df_combo.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_all.pkl'))